# Introduction

## Objectif du Notebook

L'objectif de ce notebook est de développer et d'évaluer des modèles d'analyse de sentiments en utilisant **BERT** (Bidirectional Encoder Representations from Transformers), tout en comparant différentes versions des données textuelles : avec et sans pré-traitements (lemmatisation et stemming). BERT, étant un modèle pré-entraîné puissant, nous permet de capturer les relations contextuelles entre les mots dans un texte. En parallèle, nous allons comparer les performances des modèles CNN et LSTM entraînés sur des textes bruts et pré-traités.

## Présentation de la Méthode BERT

BERT est un modèle de traitement du langage naturel basé sur une architecture de **Transformers**, permettant de capturer de manière bidirectionnelle les relations entre les mots. BERT est particulièrement efficace pour des tâches telles que l'analyse de sentiments car il tient compte du contexte des mots dans les phrases, contrairement aux modèles classiques comme Word2Vec qui sont unidirectionnels.

### BERT (Bidirectional Encoder Representations from Transformers)

BERT prend en compte le contexte avant et après un mot pour créer des représentations contextuelles riches. C'est pourquoi il est très performant pour des tâches de compréhension sémantique, comme la classification de sentiments, où l'interprétation d'un mot dépend de son contexte global.

### Comparaison avec et sans Pré-traitements (Lemmatisation et Stemming)

Dans ce notebook, nous utiliserons à la fois :
- **Les textes pré-traités** (lemmatisation et stemming) : Ces techniques consistent à ramener les mots à leur forme de base ou à leur racine pour réduire la variabilité linguistique. Cela permet de simplifier les phrases avant de les soumettre à BERT.
- **Les textes bruts** : Sans aucun pré-traitement linguistique, laissant à BERT la capacité d'analyser les mots dans leur forme originale et de capturer les nuances du langage.

L'objectif est de comparer l'impact du pré-traitement sur les performances du modèle BERT.

## Plan du Notebook

1. **Chargement et Préparation des Données** : Nous allons charger les données, les nettoyer, les tokeniser, et les préparer sous trois formats : brut, lemmatisé et stemmé.
2. **Tokenisation avec BERT** : Nous utiliserons le tokenizer BERT pour convertir les trois versions de textes en tokens adaptés au modèle BERT.
3. **Utilisation de BERT** : Nous extrairons des embeddings contextuels à partir du modèle BERT pour chaque version des données.
4. **Construction des Modèles CNN et LSTM** : Nous entraînerons deux types de modèles (CNN et LSTM) sur les embeddings extraits de BERT pour chaque version de données.
5. **Entraînement des Modèles** : Nous entraînerons les modèles sur les différentes versions des embeddings BERT.
6. **Comparaison des Modèles** : Nous comparerons les performances des modèles CNN et LSTM avec les embeddings BERT pour les données brutes, lemmatisées et stemmées.
7. **Export du Meilleur Modèle** : Nous exporterons le modèle le plus performant pour une utilisation future dans une API de prédiction.


# 1. Importation des Bibliothèques

In [46]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, TFBertModel


# 2. Chargement des Données

In [49]:
# Charger les données (en supposant que vous avez déjà les colonnes nettoyées)
data = pd.read_csv('../data/database_p7_rework.csv')

# Transformation des labels : 0 reste 0 et 4 devient 1
data['target_binary'] = data['target'].apply(lambda x: 0 if x == 0 else 1)

# Vérification des transformations
print(data['target_binary'].value_counts())

# Ensuite, définissez y comme suit :
y = data['target_binary']

0    800000
1    798315
Name: target_binary, dtype: int64


# 3. Préparation des Données
Tokenization avec BERT

In [79]:
from transformers import BertTokenizer

# Initialisation du tokenizer BERT pré-entraîné pour les textes en minuscules et sans caractères spéciaux
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour préparer les séquences BERT
def prepare_sequences_bert(texts, tokenizer, maxlen=100):
    return tokenizer(texts.tolist(), padding='max_length', truncation=True, max_length=maxlen, return_tensors='tf')

# Sélection d'un échantillon équilibré de 16 000 données (8 000 par classe)
sample_data = data.groupby('target_binary', group_keys=False).apply(lambda x: x.sample(8000, random_state=42))

# Préparation des séquences avec une longueur max de 100
X_cleaned_sample = prepare_sequences_bert(sample_data['text_cleaned'], tokenizer, maxlen=100)

# Cible (target) - identique pour toutes les variantes
y_sample = sample_data['target_binary'].values

print("Données préparées avec succès.")


D:\Master_Openclassroom\python\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Données préparées avec succès.


In [81]:
from transformers import TFBertModel

# Charger le modèle BERT pré-entraîné
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

print("Modèle BERT chargé avec succès.")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Modèle BERT chargé avec succès.


In [83]:
import numpy as np

# Générateur de batchs pour BERT
def bert_embedding_generator(sequences, labels, attention_masks, batch_size=32):
    while True:
        for i in range(0, len(sequences), batch_size):
            batch_sequences = sequences[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            batch_attention_masks = attention_masks[i:i + batch_size]
            yield {
                'input_ids': batch_sequences, 
                'attention_mask': batch_attention_masks
            }, batch_labels  # Retourner les séquences, masques d'attention et labels


In [85]:
import tensorflow as tf

# Fonction pour obtenir les embeddings BERT par batchs
def get_bert_embeddings_in_batches(generator, bert_model):
    embeddings = []
    for batch_data, batch_labels in generator:
        batch_input_ids = batch_data['input_ids']
        batch_attention_mask = batch_data['attention_mask']
        
        # Générer les embeddings pour chaque batch
        batch_embeddings = bert_model(input_ids=batch_input_ids, attention_mask=batch_attention_mask).last_hidden_state
        embeddings.append(batch_embeddings)

    return tf.concat(embeddings, axis=0)


In [87]:
from sklearn.model_selection import train_test_split

# Récupérer les input_ids et attention_mask des séquences préparées
X_cleaned_sample_input_ids = X_cleaned_sample['input_ids'].numpy()  # Convertir en numpy
X_cleaned_sample_attention_mask = X_cleaned_sample['attention_mask'].numpy()  # Convertir en numpy

# Split en données d'entraînement et de test avec stratification
X_train_cleaned_ids, X_test_cleaned_ids, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned_sample_input_ids, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

X_train_mask, X_test_mask = train_test_split(
    X_cleaned_sample_attention_mask, test_size=0.2, random_state=42, stratify=y_sample)

print("Données scindées en ensemble d'entraînement et de test.")


Données scindées en ensemble d'entraînement et de test.


In [ ]:
# Créer les générateurs
train_generator = bert_embedding_generator(X_train_cleaned_ids, y_train_cleaned, X_train_mask, batch_size=32)
test_generator = bert_embedding_generator(X_test_cleaned_ids, y_test_cleaned, X_test_mask, batch_size=32)

# Générer les embeddings pour les ensembles d'entraînement et de test
X_train_embeddings = get_bert_embeddings_in_batches(train_generator, bert_model)
X_test_embeddings = get_bert_embeddings_in_batches(test_generator, bert_model)

print("Embeddings BERT générés pour les ensembles d'entraînement et de test.")


# Construction des modèles CNN et LSTM avec BERT

In [ ]:
import mlflow
import mlflow.keras
import time
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid

def create_cnn_model(max_length=100, num_filters=128, kernel_size=5, dropout_rate=0.2, activation_type='relu'):
    model = Sequential()

    # Couche de convolution et normalisation
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(max_length, 300)))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    
    # Ajout de l'activation dynamique dans les couches denses
    if activation_type == 'relu':
        model.add(Dense(128, activation='relu'))
    elif activation_type == 'leaky_relu':
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.1))
    elif activation_type == 'prelu':
        model.add(Dense(128))
        model.add(PReLU())

    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model



# Fonction pour entraîner et loguer un modèle CNN avec GridSearch et les activations avancées
def train_and_log_cnn(X_train, y_train, X_test, y_test, experiment_name, param_grid, max_length=100):
    mlflow.set_experiment(experiment_name)
    
    best_model = None
    best_accuracy = 0
    best_params = None

    # Parcourir chaque combinaison d'hyperparamètres
    for params in ParameterGrid(param_grid):
        num_filters = params['num_filters']
        kernel_size = params['kernel_size']
        dropout_rate = params['dropout_rate']
        activation_type = params['activation_type']

        with mlflow.start_run(run_name=f"CNN_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}_activation={activation_type}"):

            # Créer le modèle CNN avec les hyperparamètres courants
            model = create_cnn_model(max_length=max_length, num_filters=num_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, activation_type=activation_type)

            # Early stopping pour éviter l'overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=4)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

            # Entraîner le modèle
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=1)
            training_time = time.time() - start_time

            # Prédictions et évaluation
            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            y_pred_proba = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc_score = roc_auc_score(y_test, y_pred_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            # Matrice de confusion
            conf_matrix = confusion_matrix(y_test, y_pred)

            # Loguer les hyperparamètres dans MLFlow
            mlflow.log_param("num_filters", num_filters)
            mlflow.log_param("kernel_size", kernel_size)
            mlflow.log_param("dropout_rate", dropout_rate)
            mlflow.log_param("activation_type", activation_type)

            # Loguer les métriques dans MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("auc", auc_score)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("training_time", training_time)

            # Sauvegarder le modèle avec MLFlow
            mlflow.keras.log_model(model, f"cnn_model_{num_filters}_{kernel_size}_{dropout_rate}")

            # Sauvegarder et loguer la matrice de confusion
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.xlabel('Prédictions')
            plt.ylabel('Vérités')
            plt.title(f"Matrice de Confusion - CNN")
            conf_matrix_path = f"./matrice/confusion_matrix_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(conf_matrix_path)
            mlflow.log_artifact(conf_matrix_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Sauvegarder et loguer la courbe ROC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            plt.figure()
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_score:.2f})")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend(loc="best")
            roc_curve_path = f"./matrice/roc_curve_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(roc_curve_path)
            mlflow.log_artifact(roc_curve_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Comparer pour garder le meilleur modèle
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
                best_model = model

    print(f"Meilleurs paramètres : {best_params} avec une accuracy de {best_accuracy:.4f}")

    # Retourner le meilleur modèle
    return best_model, best_params



# GridSearch pour le modèle CNN avec les hyperparamètres supplémentaires
param_grid_cnn = {
    'num_filters': [64, 128, 256],   # Filtres à tester
    'kernel_size': [3, 5],        # Tailles de kernel à tester
    'dropout_rate': [0.2, 0.5],      # Dropout à tester
    'activation_type': ['relu', 'leaky_relu', 'prelu']  # Activations à tester
}

In [ ]:
# Entraîner le modèle avec les données correctement formatées
best_cnn_model_bert, best_params_bert = train_and_log_cnn(
    X_train_bert, y_train_bert, X_test_bert, y_test_bert, 
    experiment_name="CNN_bert", param_grid=param_grid_cnn, max_length=100
)

In [341]:
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential

# Fonction pour créer un modèle LSTM sans la couche d'embedding, car les embeddings BERT sont déjà calculés
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, dropout=0.3, recurrent_dropout=0.3, input_shape=input_shape)))  # LSTM bidirectionnel
    model.add(BatchNormalization())  # Ajout de BatchNorm
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))  # Augmenter Dropout
    model.add(Dense(64, activation='relu'))  # Couche Dense supplémentaire
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


# 4 Evaluation des modèles avec MLflow

In [218]:
# Définir le dossier racine où vous voulez enregistrer les artefacts MLflow
base_dir = os.path.join("..", "mlruns")

# Fonction pour configurer MLflow pour chaque modèle
def configure_mlflow(model_name):
    mlflow_base_dir = os.path.join(base_dir, model_name)
    if not os.path.exists(mlflow_base_dir):
        os.makedirs(mlflow_base_dir)
    mlflow.set_tracking_uri(f"file:///{mlflow_base_dir.replace(os.sep, '/')}")
